In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import math
import statsmodels.api as sm
import pylab as py
import random
from scipy.stats import skewnorm,t,binom,chi2,poisson,expon

In [2]:
bp_data = pd.read_csv('Bp.csv')
islander_data = pd.read_csv('Islander_data.csv')
Mental_health_data = pd.read_csv('Mental_health.csv')

(Use Bp data)
Q1 : Perform Central Limit Theorem on "Physical Activity" column. Choose,
Number of samples = 10000,
A sample size =20,
Metric : Mean
Take samples without replacement
What is the standard error after sampling procedure?

In [6]:
print(bp_data.head())
bp_data.columns

   Patient_Number  Blood_Pressure_Abnormality  Level_of_Hemoglobin  \
0               1                           1                11.28   
1               2                           0                 9.75   
2               3                           1                10.79   
3               4                           0                11.00   
4               5                           1                14.17   

   Genetic_Pedigree_Coefficient  Age  BMI  Sex  Pregnancy  Smoking  \
0                          0.90   34   23    1        1.0        0   
1                          0.23   54   33    1        NaN        0   
2                          0.91   70   49    0        NaN        0   
3                          0.43   71   50    0        NaN        0   
4                          0.83   52   19    0        NaN        0   

   Physical_activity  salt_content_in_the_diet  alcohol_consumption_per_day  \
0              45961                     48071                          NaN   


Index(['Patient_Number', 'Blood_Pressure_Abnormality', 'Level_of_Hemoglobin',
       'Genetic_Pedigree_Coefficient', 'Age', 'BMI', 'Sex', 'Pregnancy',
       'Smoking', 'Physical_activity', 'salt_content_in_the_diet',
       'alcohol_consumption_per_day', 'Level_of_Stress',
       'Chronic_kidney_disease', 'Adrenal_and_thyroid_disorders'],
      dtype='object')

In [14]:
data_physical_activity = bp_data['Physical_activity']
print(data_physical_activity)
population = data_physical_activity.tolist()
print(population)
print(len(population))
samples = []
for i in range(10000):
    t = np.random.choice(population, 20, replace=False)
    print(t)
    samples.append(t)
sample_means = np.mean(samples, axis=1)
std_error = np.std(population) / np.sqrt(20)
print("Standard Error:", std_error)

[45961, 26106, 9995, 10635, 15619, 27042, 38369, 29781, 814, 1278, 48527, 22500, 25639, 11750, 14562, 7329, 29834, 49430, 43934, 25466, 14976, 41119, 19422, 29724, 4861, 48477, 22277, 44478, 30738, 24834, 3922, 11788, 27985, 27249, 26185, 14798, 21038, 2907, 4854, 38123, 32576, 38769, 12184, 21226, 23274, 9982, 48799, 39998, 32570, 23703, 5115, 14132, 33128, 23921, 23238, 38891, 49528, 18786, 38001, 14479, 29227, 17684, 43708, 23053, 40046, 43390, 44569, 40441, 42933, 31640, 22390, 10038, 45191, 27195, 3260, 10122, 45920, 43215, 4337, 46941, 17909, 12023, 12842, 33992, 25535, 33910, 37930, 42064, 37256, 35393, 39603, 45178, 32759, 17136, 39948, 43882, 4212, 40317, 22275, 31104, 5404, 37973, 829, 7889, 4697, 40537, 35904, 10540, 19499, 36148, 39575, 38296, 14420, 32753, 20090, 13771, 13785, 39419, 1708, 32968, 27466, 26972, 26209, 41201, 8334, 31093, 11026, 39556, 9710, 13933, 33938, 42494, 47979, 15640, 23294, 44799, 25633, 40321, 28096, 26445, 31317, 8836, 37776, 9893, 27833, 46510, 3

(Use Bp data)
Q2 : What is "salt_content_in_the_diet" range such that doctor 95% confident that the patient have BP? Use "Bootstrap" and normalize "salt_content_in_the_diet" data before computing confidence interval.

In [35]:
salt_content_data = bp_data['salt_content_in_the_diet']
print(salt_content_data)
n_iterations = 1000
sample_size = 20
bootstrap_means = []
for _ in range(n_iterations):
    bootstrap_sample = np.random.choice(salt_content_data, size=sample_size, replace=True)
    bootstrap_means.append(np.mean(bootstrap_sample))
normalized_data = (salt_content_data - np.mean(salt_content_data)) / np.std(salt_content_data)
confidence_interval = np.percentile(normalized_data, [2.5, 97.5])
print("95% Confidence Interval for Salt Content in the Diet:", confidence_interval)


0       48071
1       25333
2       29465
3        7439
4       49644
        ...  
1995    47568
1996     8063
1997    24753
1998    15275
1999    30463
Name: salt_content_in_the_diet, Length: 2000, dtype: int64
95% Confidence Interval for Salt Content in the Diet: [-1.66418323  1.66186104]


(Use Mental_health data)
- Replace "Have you ever sought treatment for a mental health disorder from a mental health professional?" with "professional_help".
- Replace "What is your gender?" with "gender".
- Ignore other columns of the data

Q3: Create Contingency Table "professional_help" (as index) and "gender" (columns). How many female seeks professional help?

In [19]:
print(Mental_health_data.columns)

Index(['#', '<strong>Are you self-employed?</strong>',
       'How many employees does your company or organization have?',
       'Is your employer primarily a tech company/organization?',
       'Is your primary role within your company related to tech/IT?',
       'Does your employer provide mental health benefits as part of healthcare coverage?',
       'Do you know the options for mental health care available under your employer-provided health coverage?',
       'Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?',
       'Does your employer offer resources to learn more about mental health disorders and options for seeking help?',
       'Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?',
       ...
       'What is your gender?', 'What country do you <strong>live</strong> in?',
       'What US state or t

In [23]:
Mental_health_data.rename(columns={'Have you ever sought treatment for a mental health disorder from a mental health professional?': 'professional_help'}, inplace=True)
Mental_health_data.rename(columns={'What is your gender?': 'gender'}, inplace=True)
# print(Mental_health_data)
Data_after_ignoring_other_column = pd.concat([Mental_health_data['professional_help'], Mental_health_data['gender']], axis=1)
print(Data_after_ignoring_other_column)


     professional_help  gender
0                    1  Female
1                    0    male
2                    1    male
3                    1    Male
4                    1  female
..                 ...     ...
751                  1    Male
752                  1    Male
753                  1     NaN
754                  0    dude
755                  1     NaN

[756 rows x 2 columns]


In [24]:
contingency_table = pd.crosstab(Data_after_ignoring_other_column['professional_help'], Data_after_ignoring_other_column['gender'])
print(contingency_table)

gender             Agender/genderfluid  Cis Male  Cis male  Cis-male  \
professional_help                                                      
0                                    1         0         1         0   
1                                    0         1         0         2   

gender             Contextual   F  F, cisgender  Female  Female   \
professional_help                                                  
0                           1   4             0      34        0   
1                           0  12             1      74        3   

gender             Female (cis)   ...   m  male  \
professional_help                 ...             
0                              0  ...  10    56   
1                              1  ...  13    46   

gender             male (hey this is the tech industry you're talking about)  \
professional_help                                                              
0                                                                  1     

In [25]:
print(contingency_table.columns)

Index(['Agender/genderfluid', 'Cis Male', 'Cis male', 'Cis-male', 'Contextual',
       'F', 'F, cisgender', 'Female', 'Female ', 'Female (cis) ', 'Female-ish',
       'Genderfluid', 'Genderqueer demigirl', 'Genderqueer/non-binary',
       'God King of the Valajar', 'M', 'Mail', 'Male', 'Male ', 'Male (cis)',
       'Male, cis', 'Male-ish', 'Man', 'My sex is female.', 'Non binary',
       'Non-binary', 'Nonbinary', 'Transfeminine', 'Woman', 'Woman-identified',
       '\-', 'cis female', 'cis hetero male', 'cis male ', 'cis-Female',
       'cis-male', 'dude', 'f', 'femail', 'female', 'female (cis)',
       'female (cisgender)', 'femalw', 'm', 'male',
       'male (hey this is the tech industry you're talking about)',
       'male/androgynous ', 'man', 'nonbinary', 'sometimes', 'trans woman',
       'uhhhhhhhhh fem genderqueer?', 'woman'],
      dtype='object', name='gender')


In [26]:
filtered_columns = contingency_table.filter(like='F', axis=1)
print(filtered_columns)


gender              F  F, cisgender  Female  Female   Female (cis)   \
professional_help                                                     
0                   4             0      34        0              0   
1                  12             1      74        3              1   

gender             Female-ish  cis-Female  
professional_help                          
0                           0           0  
1                           1           1  


In [27]:
filtered_columns = contingency_table.filter(like='f', axis=1)
print(filtered_columns)

gender             Agender/genderfluid  Genderfluid  God King of the Valajar  \
professional_help                                                              
0                                    1            0                        1   
1                                    0            1                        0   

gender             My sex is female.  Transfeminine  Woman-identified  \
professional_help                                                       
0                                  0              0                 0   
1                                  1              1                 1   

gender             cis female   f  femail  female  female (cis)  \
professional_help                                                 
0                           0   3       0      17             0   
1                           1  10       1      46             1   

gender             female (cisgender)  femalw  uhhhhhhhhh fem genderqueer?  
professional_help                       

Perform Chi-square test on contingency table created in above question using chi2_contingency (in_built function). What is chi-square value?

In [28]:
from scipy.stats import chi2_contingency
chi2_stat, p_val, dof, expected = chi2_contingency(contingency_table)
print("Chi-square value:", chi2_stat)


Chi-square value: 65.49410937371655


In [37]:
from scipy.stats import chi2_contingency
chi2_stat, p_val, dof, expected = chi2_contingency(contingency_table)

observed = 183
expected_male = expected[0][1]  
pearson_residual_male = (observed - expected_male) / (expected_male**0.5)

print("Pearson Residual for male subjects who choose professional help:", pearson_residual_male)


Pearson Residual for male subjects who choose professional help: 288.13150475669624
